# Backtest Execution

Run a backtest for a single strategy and view results.

**Version**: v1.11.0  
**Architecture**: Modular (lib/backtest/, lib/config/, lib/paths/, lib/bundles/, lib/validation/, lib/calendars/)

## Configuration

Set the strategy name and date range below.

**Note**: This notebook uses the v1.11.0 modular architecture. All imports use canonical paths from `lib/` packages.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'  # Change this to your strategy name
start_date = '2020-01-01'  # Start date (YYYY-MM-DD)
end_date = None  # None = today
capital_base = None  # None = from config
bundle = None  # None = auto-detect
asset_class = None  # None = auto-detect


In [ ]:
# Setup: Add project root to path
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Standard library imports
import json

# Third-party imports
import pandas as pd

# Local imports - lib modules (v1.11.0 modular architecture)
# All imports use canonical paths for consistency and future compatibility
from lib.paths import get_project_root, get_results_dir
from lib.backtest import run_backtest, save_results
from lib.config import load_strategy_params
from lib.validation import validate_bundle
from lib.calendars import get_calendar_for_asset_class


In [ ]:
# Load strategy parameters
try:
    params = load_strategy_params(strategy_name, asset_class)
    print(f"✓ Loaded parameters for {strategy_name}")
    
    # Extract asset class and bundle info
    asset_class = params.get('strategy', {}).get('asset_class', 'equities')
    print(f"✓ Asset class: {asset_class}")
    
    # Get calendar for asset class
    calendar_name = get_calendar_for_asset_class(asset_class)
    print(f"✓ Trading calendar: {calendar_name}")
    
except FileNotFoundError as e:
    print(f"✗ Error: {e}")
    print(f"  Ensure strategy exists in strategies/{asset_class or 'equities'}/{strategy_name}/")
    raise
except Exception as e:
    print(f"✗ Error loading parameters: {e}")
    raise


In [ ]:
# Bundle and Calendar Information
from lib.bundles import list_bundles, get_bundle_symbols
from lib.calendars import get_calendar_for_asset_class
from lib.validation import validate_bundle

# Get asset class and calendar
asset_class = params.get('strategy', {}).get('asset_class', 'equities')
calendar_name = get_calendar_for_asset_class(asset_class)
print(f"✓ Trading calendar: {calendar_name}")

# List available bundles
print("\nAvailable bundles:")
available_bundles = list_bundles()
for bundle_name in sorted(available_bundles)[:10]:  # Show first 10
    print(f"  - {bundle_name}")

# Display bundle information if specified
if bundle:
    try:
        symbols = get_bundle_symbols(bundle)
        print(f"\n✓ Bundle '{bundle}' contains {len(symbols)} symbols")
        if len(symbols) <= 10:
            print(f"  Symbols: {', '.join(symbols)}")
        else:
            print(f"  Symbols (first 10): {', '.join(symbols[:10])}...")
    except Exception as e:
        print(f"⚠ Could not load bundle info: {e}")

In [ ]:
# Validate bundle if specified
if bundle:
    print(f"\nValidating bundle: {bundle}")
    try:
        validation_result = validate_bundle(bundle)
        if validation_result.is_valid:
            print(f"  ✓ Bundle validation passed")
        else:
            print(f"  ⚠ Bundle validation issues found:")
            for issue in validation_result.issues[:3]:  # Show first 3
                print(f"    - {issue.message}")
    except Exception as e:
        print(f"  ⚠ Bundle validation error: {e}")

# Run backtest
print(f"Running backtest for {strategy_name}...")
print(f"Date range: {start_date} to {end_date or 'today'}")
print(f"Bundle: {bundle or 'auto-detect'}")

try:
    perf = run_backtest(
        strategy_name=strategy_name,
        start_date=start_date,
        end_date=end_date,
        capital_base=capital_base,
        bundle=bundle,
        asset_class=asset_class
    )
    print("✓ Backtest complete")
except Exception as e:
    print(f"✗ Backtest failed: {e}")
    print(f"  Check that bundle exists and strategy is valid")
    raise


In [ ]:
# Save results
result_dir = save_results(
    strategy_name=strategy_name,
    perf=perf,
    params=params,
    result_type='backtest'
)

print(f"✓ Results saved to: {result_dir}")
print(f"  Absolute path: {result_dir.resolve()}")

# Verify results directory structure
results_base = get_results_dir() / strategy_name
print(f"  Results base: {results_base}")

In [ ]:
# Display metrics
metrics_file = result_dir / 'metrics.json'
with open(metrics_file) as f:
    metrics = json.load(f)

print("=" * 60)
print("PERFORMANCE METRICS")
print("=" * 60)
print(f"Total Return: {metrics.get('total_return', 0):.2%}")
print(f"Annual Return: {metrics.get('annual_return', 0):.2%}")
print(f"Sharpe Ratio: {metrics.get('sharpe', 0):.3f}")
print(f"Sortino Ratio: {metrics.get('sortino', 0):.3f}")
print(f"Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
print(f"Calmar Ratio: {metrics.get('calmar', 0):.3f}")
print(f"Annual Volatility: {metrics.get('annual_volatility', 0):.2%}")

if 'trade_count' in metrics and metrics['trade_count'] > 0:
    print(f"\nTrade Metrics:")
    print(f"  Trade Count: {metrics.get('trade_count', 0)}")
    print(f"  Win Rate: {metrics.get('win_rate', 0):.2%}")
    print(f"  Profit Factor: {metrics.get('profit_factor', 0):.3f}")
print("=" * 60)


In [ ]:
# Display equity curve
from IPython.display import Image, display

equity_curve = result_dir / 'equity_curve.png'
if equity_curve.exists():
    display(Image(str(equity_curve)))
else:
    print("Equity curve plot not found")
